In [1]:
import pandas as pd
import numpy as np
import warnings
import pattern_detection_bg_func as bg
warnings.filterwarnings("ignore")

### driver function for cluster_matching
def dtw_clustering(self):
    tmp_list = bg.cluster_matching(ref_seq[int(self[0])], ref_seq[int(self[1])], int(self[0]), int(self[1]), accept_range, accept_ratio)
    return tmp_list

### driver function for parallel computing of suspicious_clustering
def dtw_suspicious_clustering(self):
    tmp_list = bg.scan_suspicious_clusters(result_mapping, int(self[0]),ref_seq, accept_fuzzy_range, accept_fuzzy_ratio)
    return tmp_list

In [51]:
ts_data = pd.read_csv('data/som_clustering_dataset1.csv', sep = ',')
len_of_ref_seq = len(ts_data)
accept_range = 3
accept_ratio = 0.6

In [52]:
### step 1, normalize a numerical series to a given range
def norm_numerical_ts(data):
    min_max_scaler = bg.preprocessing.MinMaxScaler(feature_range=(0.01, 0.99))

    for i in data.columns[0:-1]:
        x = data[i].values.reshape(-1, 1) 
        x_scaled = min_max_scaler.fit_transform(x)
        data[i] = x_scaled
    return data

In [53]:
# step 2, transform the scaled numerical series to an alphabetical sequence
def alphabetical_seq(data):     
    seq_dict = {0:'a',1:'b',2:'c',3:'d',4:'e',5:'f',6:'g',7:'h',8:'i',9:'j',10:'k',11:'l',
                12:'m',13:'n',14:'o',15:'p',16:'q',17:'r',18:'s',19:'t',20:'u',21:'v',22:'w',23:'x',24:'y',25:'z'}
    ref_seq = [] ### for reference sequence
    arr = '' 
    slot_num = 26
    # Put array elements in different alphabet 
    for row in range(len(data)):
        for col in range(len(data.columns[0:-1])):
            arr = arr + seq_dict[int(slot_num*data.iloc[row,col])]
        ref_seq.append(arr)   
        arr = ''
    return ref_seq
norm_ts_data = norm_numerical_ts(ts_data)
ref_seq = alphabetical_seq(norm_ts_data)

In [54]:
# step 3, match the alphabetical sequence to the reference alphabetical sequence
### parallel_ck 負責控制共有哪些tasks需讓multiprocessing完成，例如共需45次兩兩波形計算
parallel_ck = bg.build_triangular_mx(len_of_ref_seq) 
dist_list = bg.imap_unordered_bar(dtw_clustering, parallel_ck, 3)
result_mapping = bg.comparison_list(len_of_ref_seq) ### save results to a nested list
for i in dist_list:
    if(len(i) > 0):  ### to skip null list: []
        result_mapping[int(i[0][0])].append([i[0][1],i[0][2]])
        result_mapping[int(i[1][0])].append([i[1][1],i[1][2]])
        

54615it [00:27, 2019.71it/s]         


In [55]:
# step 4, get target_ts's similar ts pattern cluster
accept_fuzzy_range = 3
accept_fuzzy_ratio = 0.6
parallel_ct = np.zeros((len_of_ref_seq,2))
parallel_ct[:,0] = np.arange(len_of_ref_seq)

suspicious_cluster = bg.imap_unordered_bar(dtw_suspicious_clustering, parallel_ct, 3)
suspicious_cluster.sort(key=bg.takeFirst, reverse=False)

331it [00:36,  7.89it/s]                     


In [78]:
suspicious_cluster[33]

[33,
 [[32, 0.7], [22, 0.66], [36, 0.63], [169, 0.63]],
 [3,
  4,
  5,
  6,
  8,
  9,
  12,
  14,
  15,
  16,
  17,
  20,
  22,
  24,
  36,
  51,
  59,
  169,
  186,
  191,
  286,
  292]]

In [79]:
accept_fuzzy_range = 3
accept_fuzzy_ratio = 0.6
target_ts = 33
bg.scan_suspicious_clusters(result_mapping, target_ts, ref_seq, accept_fuzzy_range, accept_fuzzy_ratio)

[33,
 [[32, 0.7], [22, 0.66], [36, 0.63], [169, 0.63]],
 [3,
  4,
  5,
  6,
  8,
  9,
  12,
  14,
  15,
  16,
  17,
  20,
  22,
  24,
  36,
  51,
  59,
  169,
  186,
  191,
  286,
  292]]

################################################################################################################################